In [12]:
!pip install trackpy
import trackpy


In [36]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.ndimage
from scipy.stats import iqr
from ipywidgets import interact
import os
from skimage.io import imread
from skimage.filters import threshold_otsu
from skimage.restoration import rolling_ball
from skimage.measure import label, regionprops
from pandas import DataFrame as df
sns.set_style('dark', rc={'image.cmap':'inferno'})

In [17]:
image_names=os.listdir('../Data/41/')
fluorescent_image_names=[el for el in image_names if 'w3RFPled' in el]
ordering=np.argsort([int(el.split('_t')[1][:-4]) for el in fluorescent_image_names])
ordered_fluo_imgs = ['../Data/41/'+fluorescent_image_names[i] for i in ordering]


In [20]:
@interact
def preprocess(i=(0,100)):
    fig,ax=plt.subplots(1,2,figsize=(10,5))
    image = imread(ordered_fluo_imgs[int(i)])
    corrected_image = image - rolling_ball(image,radius=20)
    ax[0].imshow(image)
    
    #preprocess
    thresholded=corrected_image>threshold_otsu(corrected_image)
    
    ax[1].imshow(thresholded,cmap='Greys_r')
    ax[1].set_title('N_objects= '+str(len(regionprops(label(thresholded)))))

interactive(children=(IntSlider(value=50, description='i'), Output()), _dom_classes=('widget-interact',))

In [61]:
def make_mask(image):
    corrected_image = image - rolling_ball(image,radius=20)
    thresholded=corrected_image>threshold_otsu(corrected_image)        
    return thresholded
frames=[make_mask(imread(image_name)) for image_name in ordered_fluo_imgs[:20]]

In [64]:
f = trackpy.locate(frames[0], 11, invert=True) #this is terrible
f

,y,x,mass,size,ecc,signal,raw_mass,ep
0,118.345665,95.032492,6.638859,2.911630,0.475456,0.263787,66.0,-0.003038
1,131.597868,95.105349,6.339987,3.035589,0.495312,0.246894,58.0,-0.001981
2,134.213178,57.060778,7.119654,3.001217,0.464534,0.248194,66.0,-0.003038
3,144.397178,50.523048,6.906545,3.183743,0.439378,0.228702,60.0,-0.002170
4,179.790133,245.929852,6.742815,3.132181,0.451954,0.215708,64.0,-0.002681
5,180.661349,258.054120,6.434847,3.121949,0.435963,0.210510,64.0,-0.002681
6,191.566600,251.985714,6.458237,3.028536,0.510362,0.246894,58.0,-0.001981
7,209.974745,270.712672,5.865690,2.958995,0.427826,0.222205,66.0,-0.003038
8,212.462744,257.622012,6.469932,3.217121,0.418891,0.207911,61.0,-0.002279
9,249.867887,483.398765,5.891679,3.111762,0.449725,0.209210,63.0,-0.002532


In [65]:
our_dataframe=df(columns=['y','x','mass','size','frame'])
for idx,thresholded_img in enumerate(frames):
    regions=regionprops(label(thresholded_img))
    for r in regions:
        x,y=r.centroid
        mass=r.area
        size=r.equivalent_diameter
        our_dataframe.loc[len(our_dataframe)]=x,y,mass,size,idx

In [69]:
tracked=trackpy.link(our_dataframe,25,memory=2)

Frame 19: 14 trajectories present.


In [77]:
trajectories=[np.asarray(tracked[tracked['particle']==i][['x','y']]) for i in range(np.max(tracked['particle']+1))]

In [94]:
from matplotlib.colors import hsv_to_rgb

colors = [hsv_to_rgb( (0.5+0.5*np.random.rand(3))) for i in range(len(trajectories))]

@interact
def plot_result(i=(0,20)):
    fig,ax=plt.subplots(1,1,figsize=(10,10))
    image = frames[i]
    ax.imshow(image)
    for traj_idx,traj in enumerate(trajectories):
        x,y=traj.T
        x=x[:i]
        y=y[:i]
        plt.plot(x,y,color=colors[traj_idx])
  

interactive(children=(IntSlider(value=10, description='i', max=20), Output()), _dom_classes=('widget-interact'…

In [87]:
colors

[array([1.        , 0.        , 0.63901161]),
 array([1.        , 0.04869887, 0.        ]),
 array([0.       , 1.       , 0.5344413]),
 array([0.92344114, 1.        , 0.        ]),
 array([1.        , 0.        , 0.28909269]),
 array([0.43703295, 0.        , 1.        ]),
 array([1.        , 0.61287277, 0.        ]),
 array([0.        , 0.25637453, 1.        ]),
 array([1.        , 0.        , 0.08456124]),
 array([0.54606564, 1.        , 0.        ]),
 array([1.        , 0.15393133, 0.        ]),
 array([0.        , 1.        , 0.52525673]),
 array([1.        , 0.        , 0.02177395]),
 array([0.74018814, 0.        , 1.        ]),
 array([1.        , 0.        , 0.60573898])]